In each cell that I could, I added some code to save some statistics in a different file so that you dont have to scroll through this notebook for them.

# Step 0 & 1: Understanding and Cleaning.

In [1]:
# Standard libraries
import random
import json
import os
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn 

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

 To run, update the second cell with a filepath and a name and RUN ALL!  
 (you need a dataset with similar shape)

In [ ]:
# Step 0: Setup and Initialization

# Create base output directory
os.makedirs('output_files', exist_ok=True)

# Define input data structure (example format)
CREATORS = []  # Add creator names here
FILEPATHS = [
                # add filepaths here
    ]

# Create creator folders
for creator in CREATORS:
    os.makedirs(f'output_files/{creator}', exist_ok=True)

# Create DataFrames dictionary to store each creator's data
dfs = {}

# Create/open the statistics file
with open('output_files/statistics.txt', 'w') as f:
    f.write("Initial Statistics:\n\n")

# Load all datasets
for creator, path in zip(CREATORS, FILEPATHS):
    try:
        # Modified this line to properly handle Windows paths
        dfs[creator] = pd.read_csv(rf'{path}')
        print(f"Successfully loaded dataset for {creator}")
        
        # Append initial statistics to the main statistics file
        with open('output_files/statistics.txt', 'a') as f:
            f.write(f"{creator} initial row count: {len(dfs[creator])}\n")
            
    except Exception as e:
        print(f"Error loading dataset for {creator}: {e}")

# Print initial statistics for each dataset
for creator, df in dfs.items():
    print(f"\nStatistics for {creator}:")
    print(f"Number of rows: {len(df)}")

In [ ]:
# Check columns in each DataFrame
for creator, df in dfs.items():
    print(f"\n=== Columns in DataFrame for {creator} ===")
    print(df.columns.tolist())
# Drop unnecesary collumns after checking for Duplicates - we ensure that we don't cause more dupes by keeping the id and the time they were created.

#### NaN's & Duplicates & Unique users!

In [ ]:
# Check NaNs and duplicates for each creator's dataset
with open('output_files/statistics.txt', 'a') as f:
    f.write("\nNaN and Duplicate Analysis:\n")
    
    for creator, df in dfs.items():
        f.write(f"\n=== STATISTICS FOR {creator} ===\n")
        
        # Count unique senders and recipients
        unique_senders = df['sender_handle'].nunique()
        unique_recipients = df['recipient_handle'].nunique()
        
        f.write(f"Number of unique senders: {unique_senders}\n")
        f.write(f"Number of unique recipients: {unique_recipients}\n")
        
        # Calculate NaN percentages
        nan_percentage = (df.isna().sum() / len(df)) * 100
        f.write("\nPercentage of NaNs per column:\n")
        for col, pct in nan_percentage.items():
            f.write(f"{col}: {pct:.2f}%\n")
        
        # Check duplicates
        duplicates = df.duplicated().sum()
        f.write(f"\nNumber of duplicate rows: {duplicates}\n")
        
        # Print to console as well
        print(f"\n=== STATISTICS FOR {creator} ===")
        print(f"Number of unique senders: {unique_senders}")
        print(f"Number of unique recipients: {unique_recipients}")
        print("\nPercentage of NaNs per column:")
        print(nan_percentage)
        print(f"\nNumber of duplicate rows: {duplicates}")
        
        if duplicates > 0:
            duplicate_rows = df[df.duplicated(keep=False)]
            print("\nDuplicate rows:")
            print(duplicate_rows)
            # Optionally we could save duplicate rows to the statistics file

#### hmm checking which and how many rows are NaN.

In [ ]:
# Check empty text rows for each creator's dataset
with open('output_files/statistics.txt', 'a') as f:
    f.write("\nEmpty Text Analysis:\n")
    
    for creator, df in dfs.items():
        empty_text_count = df['text'].isna().sum()
        empty_text_rows = df[df['text'].isna()]
        
        # Write count to statistics file
        f.write(f"\n=== {creator} ===\n")
        f.write(f"Number of rows with empty text: {empty_text_count}\n")
        
        # Print to console
        print(f"\n=== {creator} ===")
        print(f"Number of rows with empty text: {empty_text_count}")

### WE have the numbers of duplicates now. Our focus is on 3 columns; text, sender_handle and recipient_handle. We are ignoring the created_at knowing that the data is ordered in ascending order.
### At this point the duplicates should be dropped (I didn't right away knowing that it will drop later when filtering the data). 

#### More UNDERSTANDING: 

In [ ]:
# Analyze sender and recipient activity for each creator
for creator, df in dfs.items():
    print(f"\n=== Analysis for {creator} ===")
    
    # Count how many times each sender has sent a message
    sender_counts = df['sender_handle'].value_counts().reset_index()
    sender_counts.columns = ['Sender', 'Messages Sent']

    # Count how many times each recipient has received a message
    recipient_counts = df['recipient_handle'].value_counts().reset_index()
    recipient_counts.columns = ['Recipient', 'Messages Received']

    # Create a DataFrame to display counts for Creator as a sender and recipient
    creator_as_sender = sender_counts[sender_counts['Sender'] == creator]
    creator_as_recipient = recipient_counts[recipient_counts['Recipient'] == creator]

    # Combine Creator's data with overall data for easy comparison
    combined_counts = pd.DataFrame({
        'Role': [f'{creator} as Sender', f'{creator} as Recipient'],
        'Count': [creator_as_sender.iloc[0]['Messages Sent'] if not creator_as_sender.empty else 0,
                creator_as_recipient.iloc[0]['Messages Received'] if not creator_as_recipient.empty else 0]
    })

    # Append other unique senders and recipients, excluding Creator for clarity
    other_senders = sender_counts[sender_counts['Sender'] != creator]
    other_recipients = recipient_counts[recipient_counts['Recipient'] != creator]

    # Display the results
    print(f"\n{creator}'s Activity:")
    print(combined_counts)
    print("\nOther Senders' Activity:")
    print(other_senders.head(2))
    print("\nOther Recipients' Activity:")
    print(other_recipients.head(2))
    
    # Save combined counts to statistics file
    with open('output_files/statistics.txt', 'a') as f:
        f.write(f"\n=== Activity Analysis for {creator} ===\n")
        f.write(f"{creator}'s Activity:\n")
        f.write(combined_counts.to_string())
        f.write("\n")

#### Short Story: For what it's worth, during preliminary tests I had a hard time combining the 2 dataframes into a single summary csv. In the end, i just processed things differently so that the data is all in one dataframe.

### Checking rows where the 'creator' is neither the sender nor the recipient (just to be safe):

In [ ]:
# Check rows where creator is neither sender nor recipient
for creator, df in dfs.items():
    rows_without_creator = df[(df['sender_handle'] != creator) & (df['recipient_handle'] != creator)]
    print(f"\n=== {creator} ===")
    print(f"Number of rows without {creator} as sender or recipient: {rows_without_creator.shape[0]}")
    
    # Save to statistics file
    with open('output_files/statistics.txt', 'a') as f:
        f.write(f"\nRows without {creator}:\n")
        f.write(f"Number of rows without {creator} as sender or recipient: {rows_without_creator.shape[0]}\n")

### Checking if each sender is also a recipient and vice versa:

In [ ]:
# Check sender/recipient overlap for each creator
for creator, df in dfs.items():
    print(f"\n=== {creator} ===")
    
    # Create sets of all unique senders and recipients
    unique_senders = set(df['sender_handle'])
    unique_recipients = set(df['recipient_handle'])

    # Check if each sender is at least once a recipient
    senders_not_recipients = unique_senders.difference(unique_recipients)
    # Check if each recipient is at least once a sender
    recipients_not_senders = unique_recipients.difference(unique_senders)

    # Print out the results
    print("Number of senders who never received a message:", len(senders_not_recipients))
    print("Number of recipients who never sent a message:", len(recipients_not_senders))
    
    # Save to statistics file
    with open('output_files/statistics.txt', 'a') as f:
        f.write(f"\nSender/Recipient Analysis for {creator}:\n")
        f.write(f"Number of senders who never received a message: {len(senders_not_recipients)}\n")
        f.write(f"Number of recipients who never sent a message: {len(recipients_not_senders)}\n")

### Drop each unique_person that didn't send or recieve a message (because you don't have a conversation) 
#### Note: I didn't drop them immedietly - I will filter everything in one go (so also the duplicates and NaN rows).

### Although we already have the number of unique senders and recipients - to determine the number of BOTH a SENDER & RECIPIENT, a simple intersection can be made:

In [ ]:
# Check individuals who are both senders and recipients for each creator
with open('output_files/statistics.txt', 'a') as f:
    f.write("\nSender/Recipient Intersection Analysis:\n")
    
    for creator, df in dfs.items():
        # Create sets of all unique senders and recipients
        unique_senders = set(df['sender_handle'])
        unique_recipients = set(df['recipient_handle'])
        
        # Find individuals who are both senders and recipients using set intersection
        both_senders_and_recipients = unique_senders.intersection(unique_recipients)
        
        # Save and print results
        f.write(f"\n=== {creator} ===\n")
        f.write(f"Number of unique individuals who are both senders and recipients: {len(both_senders_and_recipients)}\n")
        
        print(f"\n=== {creator} ===")
        print(f"Number of unique individuals who are both senders and recipients: {len(both_senders_and_recipients)}")
        
        # Uncomment this if you want to also see the list
        # print("List of individuals who are both senders and recipients:")
        # for person in both_senders_and_recipients:
        #     print(person)

### Now dropfilter the following: 1) Duplicates 2) NaNs rows from text column 3) Unique people that are not both senders and recipients 4) unique users that didn't send AND recieve more than 3 messages. 
We want to train on actual chats so small conversations (under a few messages) would add noise to the data and we already have more than enough.
Also at this point the mass messages could be dropped - but preliminary tests showed that they are not a problem (not that many and they might get dropped now anyway - can also be dropped later). Mass messages are when the creator sends to all users at once the same message. Another reason for keeping such rows (even if it would be automatic send) is that people repsond to it so we want the model to understand the context.

In [ ]:
# Process and filter each creator's dataset
for creator, df in dfs.items():
    print(f"\n=== Processing {creator}'s dataset ===")
    
    # Store initial number of rows
    initial_rows = len(df)
    print(f"Initial number of rows: {initial_rows}")
    
    # First drop duplicates
    df = df.drop_duplicates()
    print(f"Rows after dropping duplicates: {len(df)}")
    
    # Drop unnecessary columns
    # df = df.drop(['id', 'created_at'], axis=1)
    # print("Dropped 'id' and 'created_at' columns")
    
    # Drop NaN rows from text column
    df = df.dropna(subset=['text'])
    print(f"Rows after dropping NaN: {len(df)}")
    
    # Get the intersection set for this creator's dataset
    unique_senders = set(df['sender_handle'])
    unique_recipients = set(df['recipient_handle'])
    both_senders_and_recipients = unique_senders.intersection(unique_recipients)
    
    # Filter rows where both sender and recipient are in the intersection set
    df = df[df['sender_handle'].isin(both_senders_and_recipients) & 
            df['recipient_handle'].isin(both_senders_and_recipients)]
    print(f"Rows after filtering non-conversational users: {len(df)}")
    
    # Count messages sent and received by each person
    send_counts = df['sender_handle'].value_counts()
    receive_counts = df['recipient_handle'].value_counts()
    
    # Combine send and receive counts
    counts = pd.DataFrame({
        'sent': send_counts,
        'received': receive_counts
    }).fillna(0)
    
    # Keep only users who have sent AND received more than 3 messages
    qualified_users = counts[(counts['sent'] > 3) & (counts['received'] > 3)].index
    
    # Final filtering to keep only qualified users
    filtered_df = df[df['sender_handle'].isin(qualified_users) & 
                    df['recipient_handle'].isin(qualified_users)]
    
    print(f"Final number of rows: {len(filtered_df)}")
    
    # Save filtered data
    output_path = f'output_files/{creator}/filtered_data.csv'
    filtered_df.to_csv(output_path, index=False)
    print(f"Filtered data saved to {output_path}")
    print(f"Remaining participants: {len(qualified_users)}")
    
    # Update the dictionary with filtered data
    dfs[creator] = filtered_df

### Some EDA!
#### ! Before taking a look at the data lets talk about its current shape. Messages are in chronological order already but not ordered by person (eg. if 3 people texted the creator at the same time, the messages will be one after another, instead of being separated by conversation/unique_person).

#### Lets visualise/understand what we have! Excluding the creator because it has over way more messags than the most active fan.

In [ ]:
# Analyze message counts for each creator
for creator, df in dfs.items():
    print(f"\n=== Analysis for {creator} ===")
    
    # Count how many times each person has sent and received a message after cleaning.
    messages_sent_counts = df['sender_handle'].value_counts().reset_index()
    messages_sent_counts.columns = ['unique_person', 'messages_sent']
    messages_received_counts = df['recipient_handle'].value_counts().reset_index()
    messages_received_counts.columns = ['unique_person', 'messages_received']

    # Convert the series into a DataFrame for easier manipulation and future visualizations
    df_messages_sent = pd.DataFrame(messages_sent_counts)
    df_messages_received = pd.DataFrame(messages_received_counts)

    # Get creator's statistics
    creator_sent = df_messages_sent[df_messages_sent['unique_person'] == creator]['messages_sent'].iloc[0]
    creator_received = df_messages_received[df_messages_received['unique_person'] == creator]['messages_received'].iloc[0]
    
    # Append to statistics file
    with open('output_files/statistics.txt', 'a') as f:
        f.write(f"\nCreator Activity Statistics for {creator}:\n")
        f.write(f"Messages sent by creator: {creator_sent}\n")
        f.write(f"Messages received by creator: {creator_received}\n")
        f.write(f"Total messages involving creator: {creator_sent + creator_received}\n\n")

    print("DataFrame of Messages Sent Counts:")
    print(df_messages_sent.head(2))
    print("\nDataFrame of Messages Received Counts:") 
    print(df_messages_received.head(2))

### Plots!

In [12]:
for i, (creator, df) in enumerate(dfs.items(), 1):
    # Create message count DataFrames for this creator
    messages_sent_counts = df['sender_handle'].value_counts().reset_index()
    messages_sent_counts.columns = ['unique_person', 'messages_sent']
    messages_received_counts = df['recipient_handle'].value_counts().reset_index()
    messages_received_counts.columns = ['unique_person', 'messages_received']
    
    # Create DataFrames and exclude the creator
    df_messages_sent = messages_sent_counts[messages_sent_counts['unique_person'] != creator]
    df_messages_received = messages_received_counts[messages_received_counts['unique_person'] != creator]
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    fig.suptitle(f'Message Distribution Analysis (Excluding Creator {i})', fontsize=14)
    
    # Plot messages sent
    ax1.boxplot(df_messages_sent['messages_sent'], vert=False)
    ax1.set_title('Distribution of Messages Sent')
    ax1.set_xlabel('Number of Messages')
    
    # Plot messages received
    ax2.boxplot(df_messages_received['messages_received'], vert=False)
    ax2.set_title('Distribution of Messages Received')
    ax2.set_xlabel('Number of Messages')
    
    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    # Save figure to creator's folder
    plt.savefig(f'output_files/{creator}/box_plot_message_distribution_with_outliers.png')
    plt.close()

##### Without the outlier now!

In [ ]:
for i, (creator, df) in enumerate(dfs.items(), 1):
    # Create message count DataFrames for this creator
    messages_sent_counts = df['sender_handle'].value_counts().reset_index()
    messages_sent_counts.columns = ['unique_person', 'messages_sent']
    messages_received_counts = df['recipient_handle'].value_counts().reset_index()
    messages_received_counts.columns = ['unique_person', 'messages_received']
    
    # Create DataFrames and exclude the creator
    df_messages_sent = messages_sent_counts[messages_sent_counts['unique_person'] != creator]
    df_messages_received = messages_received_counts[messages_received_counts['unique_person'] != creator]
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    fig.suptitle(f'Message Distribution Analysis (Excluding Creator {i})', fontsize=14)
    
    # Plot messages sent without outliers
    ax1.boxplot(df_messages_sent['messages_sent'], vert=False, showfliers=False)
    ax1.set_title('Distribution of Messages Sent')
    ax1.set_xlabel('Number of Messages')
    
    # Plot messages received without outliers
    ax2.boxplot(df_messages_received['messages_received'], vert=False, showfliers=False)
    ax2.set_title('Distribution of Messages Received')
    ax2.set_xlabel('Number of Messages')
    
    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

### Optional: Let's check the distribution of messages and some extra statistics!

In [ ]:
for creator, df in dfs.items():
    # Create message count DataFrames
    messages_sent_counts = df['sender_handle'].value_counts().reset_index()
    messages_sent_counts.columns = ['unique_person', 'messages_sent']
    messages_received_counts = df['recipient_handle'].value_counts().reset_index()
    messages_received_counts.columns = ['unique_person', 'messages_received']
    
    # Filter out creator
    df_messages_sent_filtered = messages_sent_counts[messages_sent_counts['unique_person'] != creator]
    df_messages_received_filtered = messages_received_counts[messages_received_counts['unique_person'] != creator]

    # Print statistics
    print(f"\n=== Distribution Statistics for {creator} ===")
    
    print("\n=== Messages Sent Distribution Info ===")
    print(f"Total unique people: {len(df_messages_sent_filtered)}")
    print(f"Total messages sent: {df_messages_sent_filtered['messages_sent'].sum()}")
    print(f"Average messages per person: {df_messages_sent_filtered['messages_sent'].mean():.2f}")
    print(f"Median messages per person: {df_messages_sent_filtered['messages_sent'].median():.2f}")
    print(f"Maximum messages sent by one person: {df_messages_sent_filtered['messages_sent'].max()}")

    print("\n=== Messages Received Distribution Info ===")
    print(f"Total unique people: {len(df_messages_received_filtered)}")
    print(f"Total messages received: {df_messages_received_filtered['messages_received'].sum()}")
    print(f"Average messages per person: {df_messages_received_filtered['messages_received'].mean():.2f}")
    print(f"Median messages per person: {df_messages_received_filtered['messages_received'].median():.2f}")
    print(f"Maximum messages received by one person: {df_messages_received_filtered['messages_received'].max()}")

In [15]:
# Save distribution statistics to the shared statistics file
with open('output_files/statistics.txt', 'a') as f:
    for creator, df in dfs.items():
        messages_sent_counts = df['sender_handle'].value_counts().reset_index()
        messages_sent_counts.columns = ['unique_person', 'messages_sent']
        messages_received_counts = df['recipient_handle'].value_counts().reset_index()
        messages_received_counts.columns = ['unique_person', 'messages_received']
        
        df_messages_sent_filtered = messages_sent_counts[messages_sent_counts['unique_person'] != creator]
        df_messages_received_filtered = messages_received_counts[messages_received_counts['unique_person'] != creator]
        
        f.write(f"\n=== Distribution Statistics for {creator} ===\n")
        f.write("\nMessages Sent Distribution Info:\n")
        f.write(f"Total unique people: {len(df_messages_sent_filtered)}\n")
        f.write(f"Total messages sent: {df_messages_sent_filtered['messages_sent'].sum()}\n")
        f.write(f"Average messages per person: {df_messages_sent_filtered['messages_sent'].mean():.2f}\n")
        f.write(f"Median messages per person: {df_messages_sent_filtered['messages_sent'].median():.2f}\n")
        f.write(f"Maximum messages sent by one person: {df_messages_sent_filtered['messages_sent'].max()}\n")
        
        f.write("\nMessages Received Distribution Info:\n")
        f.write(f"Total unique people: {len(df_messages_received_filtered)}\n")
        f.write(f"Total messages received: {df_messages_received_filtered['messages_received'].sum()}\n")
        f.write(f"Average messages per person: {df_messages_received_filtered['messages_received'].mean():.2f}\n")
        f.write(f"Median messages per person: {df_messages_received_filtered['messages_received'].median():.2f}\n")
        f.write(f"Maximum messages received by one person: {df_messages_received_filtered['messages_received'].max()}\n")

# Step 2: Extracting, ordering, concatenating chats (per unique person) !

### Down below is the script in case a specific chat is needed (from the whole dataset):

In [16]:
# # TEST SAMPLE
# file_path = r'filtered_data_guide.csv'
# df = pd.read_csv(file_path)
# desired_user = 'the_username_you_want'

# user_df = df[(df['sender_handle'] == desired_user) | (df['recipient_handle'] == desired_user)]

# new_file_path = f'chats_with_{desired_user}.csv'
# user_df.to_csv(new_file_path, index=False)

# print(f"Data involving '{desired_user}' has been saved to {new_file_path}")

In [17]:
# if you want to see how many times CREATOR has interacted with someone:
# desired_user = 'the_username_you_want'

# creator_to_user = df[(df['sender_handle'] == CREATOR) & (df['recipient_handle'] == desired_user)].shape[0]
# user_to_creator = df[(df['sender_handle'] == desired_user) & (df['recipient_handle'] == CREATOR)].shape[0]

# print(f"{CREATOR} texted {desired_user} {creator_to_user} times.")
# print(f"{desired_user} texted {CREATOR} {user_to_creator} times.")

## Order per person + Concatenate the msgs

In [ ]:
# Get unique persons for each creator
for creator, df in dfs.items():
    # Combine sender and recipient handles into one Series and exclude creator
    unique_persons = pd.concat([df['sender_handle'], df['recipient_handle']])
    unique_persons = unique_persons[unique_persons != creator].unique()
    
    # Convert to list if explicitly needed as a list
    unique_person_list = list(unique_persons)
    
    # Print the length of the list of unique persons excluding creator
    print(f"Number of unique persons excluding {creator}: {len(unique_person_list)}")

In [ ]:
# Initialize an empty DataFrame to store ordered messages
for creator, df in dfs.items():
    ordered_messages = pd.DataFrame()
    
    # Get unique persons for this creator
    unique_persons = pd.concat([df['sender_handle'], df['recipient_handle']])
    unique_persons = unique_persons[unique_persons != creator].unique()
    unique_person_list = list(unique_persons)
    
    # Iterate through each unique person and gather their messages
    for person in unique_person_list:
        person_messages = df[(df['sender_handle'] == person) | (df['recipient_handle'] == person)]
        ordered_messages = pd.concat([ordered_messages, person_messages])

    # Write the ordered messages to a new CSV file
    output_path = f'output_files/{creator}/ordered_filtered_data.csv'
    ordered_messages.to_csv(output_path, index=False)
    print(f"Ordered messages for {creator} have been saved to: {output_path}")
    
    # Update the dictionary with ordered messages
    dfs[creator] = ordered_messages

#### You now have a csv odered by person AND in chronolgical order. Next step is to concatenate consecutive messages on same row.

In [ ]:
def concatenate_messages(df):
    results = []
    current_messages = ""
    current_sender = None
    current_recipient = None

    for index, row in df.iterrows():
        # Clean up the column names
        current_text = str(row['text']).strip()

        if current_sender is None or current_recipient is None:
            # Start a new message block
            current_sender = row['sender_handle']
            current_recipient = row['recipient_handle']
            current_messages = current_text
        elif row['sender_handle'] == current_sender and row['recipient_handle'] == current_recipient:
            # Continue concatenating messages for the same sender and recipient
            current_messages += " " + current_text
        else:
            # Save the current messages before switching to a new sender or recipient
            results.append({
                'text': current_messages,
                'sender_handle': current_sender,
                'recipient_handle': current_recipient
            })
            # Reset for the next sender and recipient
            current_sender = row['sender_handle']
            current_recipient = row['recipient_handle']
            current_messages = current_text

    # Don't forget to add the last set of messages
    if current_messages:
        results.append({
            'text': current_messages,
            'sender_handle': current_sender,
            'recipient_handle': current_recipient
        })

    return pd.DataFrame(results)

# Process each creator's dataset
for creator, df in dfs.items():
    # Make sure column names are stripped of leading/trailing whitespace and are lowercased
    df.columns = df.columns.str.strip().str.lower()
    
    # Process the data to concatenate messages
    concatenated_df = concatenate_messages(df)
    
    # Save the concatenated messages to a new CSV
    output_path = f'output_files/{creator}/concat_ordered_filtered_data.csv'
    concatenated_df.to_csv(output_path, index=False)
    
    # Update the dictionary with concatenated messages
    dfs[creator] = concatenated_df
    
    print(f"Concatenated conversations for {creator} have been saved to: {output_path}")

In [ ]:
# Check number of rows in concatenated files for each creator and save to statistics file
with open('output_files/statistics.txt', 'a') as f:
    f.write("\n\nRows of data after cleaning:\n\n")
    for creator, df in dfs.items():
        num_rows = len(df)
        print(f"Number of rows in {creator}'s concatenated file: {num_rows}")
        f.write(f"{creator} rows after cleaning: {num_rows}\n")


# Step 3: SPLIT TRAIN-TEST (per random but fully conversations)

In [ ]:
def extract_test_conversations(df, creator, target_size_percentage=5):
    # Calculate target size
    total_messages = len(df)
    target_size = int(total_messages * (target_size_percentage / 100))
    
    # Get list of unique users who interacted with creator
    users = set(df[df['sender_handle'] == creator]['recipient_handle']) | \
            set(df[df['recipient_handle'] == creator]['sender_handle'])
    users = list(users)
    
    # Randomly shuffle users
    random.shuffle(users)
    
    test_df = pd.DataFrame()
    test_messages_count = 0
    selected_users = []
    
    # Extract conversations until we reach target size
    for user in users:
        # Get all messages between creator and this user
        user_convo = df[
            ((df['sender_handle'] == creator) & (df['recipient_handle'] == user)) |
            ((df['sender_handle'] == user) & (df['recipient_handle'] == creator))
        ]
        
        test_messages_count += len(user_convo)
        test_df = pd.concat([test_df, user_convo])
        selected_users.append(user)
        
        if test_messages_count >= target_size:
            break
    
    # Create training set (all remaining conversations)
    train_df = df[~df.index.isin(test_df.index)]
    
    print(f"Total messages: {total_messages}")
    print(f"Test set messages: {len(test_df)} ({(len(test_df)/total_messages)*100:.2f}%)")
    print(f"Training set messages: {len(train_df)} ({(len(train_df)/total_messages)*100:.2f}%)")
    print(f"Selected users for test set: {len(selected_users)} out of {len(users)}")
    
    return train_df, test_df, selected_users

# Process each creator's dataset
for creator, df in dfs.items():
    print(f"\n=== Processing {creator}'s dataset ===")
    
    # Split the dataset
    train_df, test_df, selected_users = extract_test_conversations(df, creator, target_size_percentage=5)
    
    # Save the splits
    train_path = f'output_files/{creator}/train_concat_ordered_filtered.csv'
    test_path = f'output_files/{creator}/test_concat_ordered_filtered.csv'
    users_path = f'output_files/{creator}/test_set_users.txt'
    
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)
    
    # Save selected users for reference
    with open(users_path, 'w') as f:
        f.write('\n'.join(selected_users))
    
    print(f"Files saved to:\n{train_path}\n{test_path}\n{users_path}")
    


In [23]:
# Save final dataset statistics before formatting for fine-tuning
with open('output_files/statistics.txt', 'a') as f:
    f.write('\n\nFinal Dataset Statistics After Train/Test Split:\n\n')
    for creator, df in dfs.items():
        train_df = pd.read_csv(f'output_files/{creator}/train_concat_ordered_filtered.csv')
        test_df = pd.read_csv(f'output_files/{creator}/test_concat_ordered_filtered.csv')
        
        # Read selected test users
        with open(f'output_files/{creator}/test_set_users.txt', 'r') as users_file:
            test_users = users_file.read().splitlines()
        
        total_messages = len(train_df) + len(test_df)
        
        f.write(f'=== {creator} ===\n')
        f.write(f'Training set size: {len(train_df)} messages ({(len(train_df)/total_messages)*100:.2f}%)\n')
        f.write(f'Test set size: {len(test_df)} messages ({(len(test_df)/total_messages)*100:.2f}%)\n')
        f.write(f'Total: {total_messages} messages\n')
        f.write(f'Selected users for test set: {len(test_users)}\n\n')

## From this point, we have a clean, structured and organized dataset that we can format for fine-tuning in the required shapes. 

# Step 4: JSONL (for fine-tuning) with Strategy Specific Format 


This is the format required for fine-tuning user-assistant pairs.

Also balancing after each strategy for more control and understanding. It could be done in one cell after all the strategies if needed.

## Model A: One user-assistant pair per row. 


##### Advantage: Will end up having more rows of training data.
##### Disadvantage: No context given.

In [ ]:
def generate_model_a_pairs(df, creator):
    jsonl_pairs = []
    
    # Process each row
    for i in range(len(df) - 1):  # Go up to second-to-last row
        current_row = df.iloc[i]
        next_row = df.iloc[i + 1]
        
        # Skip if current message is from assistant (creator)
        if current_row['sender_handle'] == creator:
            continue
            
        # Check if we have a valid user-assistant pair
        if (current_row['sender_handle'] != creator and 
            next_row['sender_handle'] == creator):
            
            pair = {
                "messages": [
                    {
                        "role": "user",
                        "content": current_row['text']
                    },
                    {
                        "role": "assistant",
                        "content": next_row['text']
                    }
                ]
            }
            jsonl_pairs.append(pair)
    
    return jsonl_pairs

def save_to_jsonl(jsonl_pairs, output_file):
    with open(output_file, 'w') as file:
        for pair in jsonl_pairs:
            file.write(json.dumps(pair) + '\n')

# Process each creator's dataset
for creator, df in dfs.items():
    # Load the training data
    train_df = pd.read_csv(f'output_files/{creator}/train_concat_ordered_filtered.csv')
    
    # Generate the pairs and save to JSONL
    pairs = generate_model_a_pairs(train_df, creator)
    output_path = f'output_files/{creator}/Strategy_A_one_pair_{creator}.jsonl'
    save_to_jsonl(pairs, output_path)
    
    print(f"Generated {len(pairs)} user-assistant pairs for {creator}")

We have some rules:
{Skip any assistant messages at the start of conversations
Automatically skip any user messages at the end of conversations (since they won't have an assistant response) }
Because for the fine-tuning only user-assistant pair are accepted. 


In [ ]:
def generate_model_a_pairs(df, creator):
    jsonl_pairs = []
    
    # Process each row
    for i in range(len(df) - 1):  # Go up to second-to-last row
        current_row = df.iloc[i]
        next_row = df.iloc[i + 1]
        
        # Skip if current message is from assistant (creator)
        if current_row['sender_handle'] == creator:
            continue
            
        # Check if we have a valid user-assistant pair
        if (current_row['sender_handle'] != creator and 
            next_row['sender_handle'] == creator):
            
            pair = {
                "messages": [
                    {
                        "role": "user",
                        "content": current_row['text']
                    },
                    {
                        "role": "assistant",
                        "content": next_row['text']
                    }
                ]
            }
            jsonl_pairs.append(pair)
    
    return jsonl_pairs

def save_to_jsonl(jsonl_pairs, output_file):
    with open(output_file, 'w') as file:
        for pair in jsonl_pairs:
            file.write(json.dumps(pair) + '\n')

# Process each creator's dataset
for creator, df in dfs.items():
    # Load the training data
    train_df = pd.read_csv(f'output_files/{creator}/train_concat_ordered_filtered.csv')
    
    # Generate the pairs and save to JSONL
    pairs = generate_model_a_pairs(train_df, creator)
    output_path = f'output_files/{creator}/Strategy_A_one_pair_{creator}.jsonl'
    save_to_jsonl(pairs, output_path)
    
    print(f"Generated {len(pairs)} user-assistant pairs for {creator}")

In [ ]:
# Find the file with the lowest number of rows
row_counts = {}
for creator in dfs.keys():
    jsonl_path = f'output_files/{creator}/Strategy_A_one_pair_{creator}.jsonl'
    with open(jsonl_path, 'r') as f:
        rows = len(f.readlines())  # Count number of lines in JSONL file
    row_counts[creator] = rows

min_rows = min(row_counts.values())
print(f"\nInitial row counts:")
for creator, count in row_counts.items():
    print(f"{creator}: {count} rows")
print(f"\nSmallest number of rows: {min_rows}")

# Save deletion statistics to file
with open('output_files/statistics.txt', 'a') as f:
    f.write("\n\nStrategy-A JSONL Row Reduction Statistics:\n")
    f.write(f"Target row count (minimum): {min_rows}\n")
    f.write("\nRows removed per creator:\n")

    # Equalize all files to the minimum number of rows
    for creator in dfs.keys():
        jsonl_path = f'output_files/{creator}/Strategy_A_one_pair_{creator}.jsonl'
        
        # Read all rows
        with open(jsonl_path, 'r') as file:
            rows = [json.loads(line) for line in file]
        
        if len(rows) > min_rows:
            rows_to_remove = len(rows) - min_rows
            # Randomly select rows to keep
            rows = random.sample(rows, min_rows)
            
            # Save back the reduced dataset
            with open(jsonl_path, 'w') as file:
                for row in rows:
                    file.write(json.dumps(row) + '\n')
            
            print(f"\n{creator}:")
            print(f"- Original rows: {row_counts[creator]}")
            print(f"- Rows removed: {rows_to_remove}")
            print(f"- Final rows: {len(rows)}")
            
            f.write(f"\n{creator}:\n")
            f.write(f"- Original rows: {row_counts[creator]}\n")
            f.write(f"- Rows removed: {rows_to_remove}\n")
            f.write(f"- Final rows: {len(rows)}\n")
        else:
            print(f"\n{creator}: No rows removed (already at minimum)")
            f.write(f"\n{creator}: No rows removed (already at minimum)\n")

print("\nRow reduction complete. All Strategy-A JSONL files now have the same number of rows.")

## Model B: From 10 to 10 message pairs on row until that specific conversation ends. 


##### Edge case 1: If a conversation has less than 20 messages - there will be only one chunk for that conversation.
##### Edge case 2: The last chunk will have less than 20 if the number of total messages is not divisible by 20.
##### Advantage: Has context.
##### Disadvantage: Data is very compressed - will have fewer training rows.

In [ ]:
def process_conversations_model_b(df, creator, max_pairs=10):
    jsonl_chunks = []
    current_convo = []
    current_user = None
    
    for _, row in df.iterrows():
        user = row['recipient_handle'] if row['sender_handle'] == creator else row['sender_handle']
        sender = row['sender_handle']
        role = 'user' if sender != creator else 'assistant'
        message = row['text']
        
        # Start new conversation if user changes
        if user != current_user:
            if current_convo:
                # Process remaining messages in previous conversation
                if len(current_convo) >= 2 and current_convo[-1]['role'] == 'assistant':
                    while current_convo and current_convo[0]['role'] == 'assistant':
                        current_convo = current_convo[1:]
                    while current_convo and current_convo[-1]['role'] == 'user':
                        current_convo = current_convo[:-1]
                    if len(current_convo) >= 2:
                        chunk = {"messages": current_convo}
                        jsonl_chunks.append(chunk)
            current_convo = []
            current_user = user
        
        current_convo.append({'role': role, 'content': message})
        
        if len(current_convo) >= max_pairs * 2:
            temp_convo = current_convo.copy()
            while temp_convo and temp_convo[0]['role'] == 'assistant':
                temp_convo = temp_convo[1:]
            while temp_convo and temp_convo[-1]['role'] == 'user':
                temp_convo = temp_convo[:-1]
            if len(temp_convo) >= 2 and temp_convo[-1]['role'] == 'assistant':
                chunk = {"messages": temp_convo}
                jsonl_chunks.append(chunk)
                current_convo = []
    
    if current_convo:
        while current_convo and current_convo[0]['role'] == 'assistant':
            current_convo = current_convo[1:]
        while current_convo and current_convo[-1]['role'] == 'user':
            current_convo = current_convo[:-1]
        if len(current_convo) >= 2 and current_convo[-1]['role'] == 'assistant':
            chunk = {"messages": current_convo}
            jsonl_chunks.append(chunk)
    
    return jsonl_chunks

# Process each creator's dataset
for creator, df in dfs.items():
    # Load the training data
    train_df = pd.read_csv(f'output_files/{creator}/train_concat_ordered_filtered.csv')
    
    # Generate the chunks and save to JSONL
    chunks = process_conversations_model_b(train_df, creator)
    output_path = f'output_files/{creator}/Strategy_B_ten_pairs_{creator}.jsonl'
    save_to_jsonl(chunks, output_path)
    
    print(f"Generated {len(chunks)} conversation chunks for {creator}")

In [ ]:
# Find the file with the lowest number of rows
row_counts = {}
for creator in dfs.keys():
    jsonl_path = f'output_files/{creator}/Strategy_B_ten_pairs_{creator}.jsonl'
    with open(jsonl_path, 'r') as f:
        rows = len(f.readlines())  # Count number of lines in JSONL file
    row_counts[creator] = rows

min_rows = min(row_counts.values())
print(f"\nInitial row counts:")
for creator, count in row_counts.items():
    print(f"{creator}: {count} rows")
print(f"\nSmallest number of rows: {min_rows}")

# Save deletion statistics to file
with open('output_files/statistics.txt', 'a') as f:
    f.write("\n\nStrategy B JSONL Row Reduction Statistics:\n")
    f.write(f"Target row count (minimum): {min_rows}\n")
    f.write("\nRows removed per creator:\n")

    # Equalize all files to the minimum number of rows
    for creator in dfs.keys():
        jsonl_path = f'output_files/{creator}/Strategy_B_ten_pairs_{creator}.jsonl'
        
        # Read all rows
        with open(jsonl_path, 'r') as file:
            rows = [json.loads(line) for line in file]
        
        if len(rows) > min_rows:
            rows_to_remove = len(rows) - min_rows
            # Randomly select rows to keep
            rows = random.sample(rows, min_rows)
            
            # Save back the reduced dataset
            with open(jsonl_path, 'w') as file:
                for row in rows:
                    file.write(json.dumps(row) + '\n')
            
            print(f"\n{creator}:")
            print(f"- Original rows: {row_counts[creator]}")
            print(f"- Rows removed: {rows_to_remove}")
            print(f"- Final rows: {len(rows)}")
            
            f.write(f"\n{creator}:\n")
            f.write(f"- Original rows: {row_counts[creator]}\n")
            f.write(f"- Rows removed: {rows_to_remove}\n")
            f.write(f"- Final rows: {len(rows)}\n")
        else:
            print(f"\n{creator}: No rows removed (already at minimum)")
            f.write(f"\n{creator}: No rows removed (already at minimum)\n")

print("\nRow reduction complete. All Strategy B JSONL files now have the same number of rows.")

## Model C: Rolling format. Starts with one message pair and incrementally adds another pair from that specific conversation until it reaches 3 pairs. Then it rolls by incrementally removing the first pair and appending the next pair until there are no other message pairs in that conversation.

##### Advantages: Simulates the natural flow of a conversation. Divesre training set.

In [ ]:
def read_csv(file_path):
    return pd.read_csv(file_path)

def process_conversations(df, creator):
    conversations = []
    current_convo = []
    current_user = None

    for _, row in df.iterrows():
        user = row['recipient_handle'] if row['sender_handle'] == creator else row['sender_handle']
        sender = row['sender_handle']
        role = 'user' if sender != creator else 'assistant'
        message = row['text']

        if user != current_user:
            if current_convo:
                conversations.append(current_convo)
            current_convo = []
            current_user = user
        
        current_convo.append({'role': role, 'content': message})

    if current_convo:
        conversations.append(current_convo)
    
    return conversations

def filter_conversation(convo):
    if convo and convo[0]['role'] == 'assistant':
        convo = convo[1:]
    if convo and convo[-1]['role'] == 'user':
        convo = convo[:-1]
    return convo

def generate_jsonl_chunks(conversation):
    jsonl_chunks = []
    start_index = 0
    chunk_size = 2
    max_pairs = 3  # Max pairs set to 3 (6 messages) - DUring preliminary tests it was even higher.

    while start_index + chunk_size <= len(conversation):
        chunk = conversation[start_index:start_index + chunk_size]
        if len(chunk) % 2 == 0:  # Ensure even number of messages
            jsonl_chunks.append({"messages": chunk})
        chunk_size += 2
        if chunk_size > max_pairs * 2:
            start_index += 2
            chunk_size = max_pairs * 2
    
    # Ensure the final chunk is the last X messages 
    remaining_chunk = conversation[-max_pairs * 2:]
    if remaining_chunk and len(remaining_chunk) % 2 == 0 and \
       (not jsonl_chunks or remaining_chunk != jsonl_chunks[-1]["messages"]):
        jsonl_chunks.append({"messages": remaining_chunk})

    return jsonl_chunks

def save_to_jsonl(jsonl_chunks, output_file):
    with open(output_file, 'w') as file:
        for chunk in jsonl_chunks:
            file.write(json.dumps(chunk) + '\n')

# Process each creator's dataset
for creator, df in dfs.items():
    print(f"\n=== Processing {creator}'s dataset ===")
    
    # Read the training data
    train_path = f'output_files/{creator}/train_concat_ordered_filtered.csv'
    df = read_csv(train_path)
    
    # Process conversations
    conversations = process_conversations(df, creator)
    print(f"Total conversations processed: {len(conversations)}")

    # Generate chunks
    all_jsonl_chunks = []
    for convo in conversations:
        filtered_convo = filter_conversation(convo)
        jsonl_chunks = generate_jsonl_chunks(filtered_convo)
        all_jsonl_chunks.extend(jsonl_chunks)

    # Save results
    output_path = f'output_files/{creator}/Strategy_C_rolling_format_{creator}.jsonl'
    save_to_jsonl(all_jsonl_chunks, output_path)
    print(f"Total chunks generated: {len(all_jsonl_chunks)}")
    print(f"Results saved to: {output_path}")

In [ ]:
# Find the file with the lowest number of rows
row_counts = {}
for creator in dfs.keys():
    jsonl_path = f'output_files/{creator}/Strategy_C_rolling_format_{creator}.jsonl'
    with open(jsonl_path, 'r') as f:
        rows = len(f.readlines())
    row_counts[creator] = rows

min_rows = min(row_counts.values())
print(f"\nInitial row counts:")
for creator, count in row_counts.items():
    print(f"{creator}: {count} rows")
print(f"\nSmallest number of rows: {min_rows}")

# Save deletion statistics to file
with open('output_files/statistics.txt', 'a') as f:
    f.write("\n\nStrategy C JSONL Row Reduction Statistics:\n")
    f.write(f"Target row count (minimum): {min_rows}\n")
    f.write("\nRows removed per creator:\n")

    # Equalize all files to the minimum number of rows
    for creator in dfs.keys():
        jsonl_path = f'output_files/{creator}/Strategy_C_rolling_format_{creator}.jsonl'
        
        # Read all rows
        with open(jsonl_path, 'r') as file:
            rows = [json.loads(line) for line in file]
        
        if len(rows) > min_rows:
            rows_to_remove = len(rows) - min_rows
            # Randomly select rows to keep
            rows = random.sample(rows, min_rows)
            
            # Save back the reduced dataset
            with open(jsonl_path, 'w') as file:
                for row in rows:
                    file.write(json.dumps(row) + '\n')
            
            print(f"\n{creator}:")
            print(f"- Original rows: {row_counts[creator]}")
            print(f"- Rows removed: {rows_to_remove}")
            print(f"- Final rows: {len(rows)}")
            
            f.write(f"\n{creator}:\n")
            f.write(f"- Original rows: {row_counts[creator]}\n")
            f.write(f"- Rows removed: {rows_to_remove}\n")
            f.write(f"- Final rows: {len(rows)}\n")
        else:
            print(f"\n{creator}: No rows removed (already at minimum)")
            f.write(f"\n{creator}: No rows removed (already at minimum)\n")

print("\nRow reduction complete. All Strategy C JSONL files now have the same number of rows.")

# Step 5: Enhance the TESTING Data.
By improving the format, we also anonymize the chat data.

#### Let's check real quick the min, average and max number of mesages per conversation in the tsting data. 
#### We will employ a similar format to the Model C/ Startegy C but with even more context. THis way we have no bias towards a given model. How? Well the test set will include rows with only one pair, with tons of pairs (and so context) and a few pairs but proper context.

In [ ]:
# Analyze test data for each creator
with open('output_files/statistics.txt', 'a') as f:
    f.write("\n\nTest Set Conversation Statistics:\n")
    
    for creator in dfs.keys():
        print(f"\n=== Test Set Analysis for {creator} ===")
        f.write(f"\n=== {creator} ===\n")
        
        # Load the test data
        test_df = pd.read_csv(f'output_files/{creator}/test_concat_ordered_filtered.csv')
        
        # Get list of unique users (excluding creator)
        unique_persons = set(test_df[test_df['sender_handle'] != creator]['sender_handle']) | \
                        set(test_df[test_df['recipient_handle'] != creator]['recipient_handle'])
        
        # Initialize dictionary to store conversation lengths
        conversation_lengths = {}
        
        # Count messages in each conversation
        for person in unique_persons:
            conversation = test_df[
                ((test_df['sender_handle'] == creator) & (test_df['recipient_handle'] == person)) |
                ((test_df['sender_handle'] == person) & (test_df['recipient_handle'] == creator))
            ]
            conversation_lengths[person] = len(conversation)
        
        # Calculate statistics
        min_messages = min(conversation_lengths.values())
        max_messages = max(conversation_lengths.values())
        avg_messages = sum(conversation_lengths.values()) / len(conversation_lengths)
        
        # Print and save statistics
        stats = [
            f"Number of unique persons in test set: {len(unique_persons)}",
            f"\nConversation Statistics:",
            f"Minimum messages of a conversation: {min_messages}",
            f"Average messages per conversation: {avg_messages:.2f}",
            f"Maximum messages of a conversation: {max_messages}"
        ]
        
        for stat in stats:
            print(stat)
            f.write(stat + '\n')

In [ ]:
def process_conversations(df, creator):
    conversations = []
    current_convo = []
    current_user = None

    for _, row in df.iterrows():
        user = row['recipient_handle'] if row['sender_handle'] == creator else row['sender_handle']
        sender = row['sender_handle']
        role = 'user' if sender != creator else 'assistant'
        message = row['text']

        if user != current_user:
            if current_convo:
                conversations.append(current_convo)
            current_convo = []
            current_user = user
        
        current_convo.append({'role': role, 'content': message})

    if current_convo:
        conversations.append(current_convo)
    
    return conversations

def filter_conversation(convo):
    # Skip assistant messages at start
    while convo and convo[0]['role'] == 'assistant':
        convo = convo[1:]
    # Skip user messages at end
    while convo and convo[-1]['role'] == 'user':
        convo = convo[:-1]
    return convo

def generate_test_jsonl_chunks(conversation):
    jsonl_chunks = []
    start_index = 0
    chunk_size = 2
    max_pairs = 6  # Set to 6 pairs (12 messages) for testing data

    while start_index + chunk_size <= len(conversation):
        chunk = conversation[start_index:start_index + chunk_size]
        if len(chunk) % 2 == 0:  # Ensure even number of messages
            jsonl_chunks.append({"messages": chunk})
        chunk_size += 2
        if chunk_size > max_pairs * 2:
            start_index += 2
            chunk_size = max_pairs * 2
    
    # Ensure the final chunk is the last 12 messages 
    remaining_chunk = conversation[-max_pairs * 2:]
    if remaining_chunk and len(remaining_chunk) % 2 == 0 and \
       (not jsonl_chunks or remaining_chunk != jsonl_chunks[-1]["messages"]):
        jsonl_chunks.append({"messages": remaining_chunk})

    return jsonl_chunks

# Process each creator's test dataset
for creator in dfs.keys():
    print(f"\n=== Processing Test Dataset for {creator} ===")
    
    # Load the test data
    test_df = pd.read_csv(f'output_files/{creator}/test_concat_ordered_filtered.csv')
    
    # Process conversations
    conversations = process_conversations(test_df, creator)
    
    all_jsonl_chunks = []
    for convo in conversations:
        filtered_convo = filter_conversation(convo)
        if filtered_convo:  # Only process if conversation is not empty after filtering
            jsonl_chunks = generate_test_jsonl_chunks(filtered_convo)
            all_jsonl_chunks.extend(jsonl_chunks)

    # Save to JSONL file
    output_path = f'output_files/{creator}/TEST_DATASET_{creator}.jsonl'
    save_to_jsonl(all_jsonl_chunks, output_path)
    
    print(f"Generated {len(all_jsonl_chunks)} test chunks for {creator}")
    print(f"Test dataset saved to: {output_path}")

In [ ]:
# Find the file with the lowest number of test chunks
row_counts = {}
for creator in dfs.keys():
    jsonl_path = f'output_files/{creator}/TEST_DATASET_{creator}.jsonl'
    with open(jsonl_path, 'r') as f:
        rows = len(f.readlines())
    row_counts[creator] = rows

min_rows = min(row_counts.values())
print(f"\nInitial test chunk counts:")
for creator, count in row_counts.items():
    print(f"{creator}: {count} chunks")
print(f"\nSmallest number of test chunks: {min_rows}")

# Save deletion statistics to file
with open('output_files/statistics.txt', 'a') as f:
    f.write("\n\nTest Dataset Row Reduction Statistics:\n")
    f.write(f"Target chunk count (minimum): {min_rows}\n")
    f.write("\nChunks removed per creator:\n")

    # Equalize all files to the minimum number of chunks
    for creator in dfs.keys():
        jsonl_path = f'output_files/{creator}/TEST_DATASET_{creator}.jsonl'
        
        # Read all chunks
        with open(jsonl_path, 'r') as file:
            chunks = [json.loads(line) for line in file]
        
        if len(chunks) > min_rows:
            chunks_to_remove = len(chunks) - min_rows
            # Randomly select chunks to keep
            chunks = random.sample(chunks, min_rows)
            
            # Save back the reduced dataset
            with open(jsonl_path, 'w') as file:
                for chunk in chunks:
                    file.write(json.dumps(chunk) + '\n')
            
            print(f"\n{creator}:")
            print(f"- Original chunks: {row_counts[creator]}")
            print(f"- Chunks removed: {chunks_to_remove}")
            print(f"- Final chunks: {len(chunks)}")
            
            f.write(f"\n{creator}:\n")
            f.write(f"- Original chunks: {row_counts[creator]}\n")
            f.write(f"- Chunks removed: {chunks_to_remove}\n")
            f.write(f"- Final chunks: {len(chunks)}\n")
        else:
            print(f"\n{creator}: No chunks removed (already at minimum)")
            f.write(f"\n{creator}: No chunks removed (already at minimum)\n")

print("\nChunk reduction complete. All test datasets now have the same number of chunks.")

# Step 6: Add the same system message to all Json files.
The models will train with a system message slightly steering them in the right directio

We are also adding a system message to the testing file. Why? When we are generating the responses, we are calling the model using the same specific system message tailored to each creator's details.

In [ ]:
# Process each creator's files (including test files)
for creator in dfs.keys():
    system_message_content = f" You are a content creator who goes by the name {creator}. You are assuming the creator's identity. " # Example
    
    # List of all files including test dataset
    all_files = [
        f'output_files/{creator}/Strategy_A_one_pair_{creator}.jsonl',
        f'output_files/{creator}/Strategy_B_ten_pairs_{creator}.jsonl', 
        f'output_files/{creator}/Strategy_C_rolling_format_{creator}.jsonl',
        f'output_files/{creator}/TEST_DATASET_{creator}.jsonl'  # Added test dataset
    ]
    
    print(f"\n=== Processing {creator}'s files ===")
    
    # Process each file
    for jsonl_file in all_files:
        print(f"Processing {jsonl_file}...")
        
        # Read the file
        with open(jsonl_file, 'r') as file:
            lines = file.readlines()
        
        # Modify each line to include system message
        modified_lines = []
        for line in lines:
            data = json.loads(line)
            # Prepend the system message if it's not already there
            if not (data['messages'] and data['messages'][0].get('role') == 'system'):
                system_message = {"role": "system", "content": system_message_content}
                data['messages'].insert(0, system_message)
            modified_lines.append(json.dumps(data))
        
        # Write back to the same file
        with open(jsonl_file, 'w') as file:
            for line in modified_lines:
                file.write(line + '\n')
        
        print(f"Added system message to {jsonl_file}")

# Step 7: Splitting the data into TRAIN - VAL.


An optional paramter while fine-tuning with OpenPipe is 'split' which can be asigned TRAIN or TEST. Like traditional ML task that use validation set to test the performance of their models, we visualy inspect the Model's output. We aknowledge that this is subjective and do not rely on it. First priority is to flag through the validation set if the Model 'behave' properly.

In [ ]:
def add_split_labels(input_file, test_percentage=1):
    # Read the JSONL file
    with open(input_file, 'r') as file:
        jsonl_data = [json.loads(line) for line in file]
    
    # Calculate number of test rows
    total_rows = len(jsonl_data)
    test_count = max(1, int(total_rows * (test_percentage / 100)))
    
    # Randomly select test indices
    test_indices = set(random.sample(range(total_rows), test_count))
    
    # Add split labels
    for i, entry in enumerate(jsonl_data):
        entry['split'] = 'TEST' if i in test_indices else 'TRAIN'
    
    # Write back to the same file
    with open(input_file, 'w') as file:
        for entry in jsonl_data:
            file.write(json.dumps(entry) + '\n')
    
    return total_rows, test_count

# Process each creator's strategy files
with open('output_files/statistics.txt', 'a') as stats_file:
    stats_file.write("\n\nSplit Labels Statistics:\n")
    
    for creator in dfs.keys():
        print(f"\n=== Processing {creator}'s Strategy Files ===")
        stats_file.write(f"\n=== {creator} ===\n")
        
        # List of strategy files for this creator
        strategy_files = [
            f'output_files/{creator}/Strategy_A_one_pair_{creator}.jsonl',
            f'output_files/{creator}/Strategy_B_ten_pairs_{creator}.jsonl',
            f'output_files/{creator}/Strategy_C_rolling_format_{creator}.jsonl'
        ]
        
        # Process each file
        for file in strategy_files:
            total_rows, test_count = add_split_labels(file)
            
            # Print and save statistics
            stats = [
                f"Processed {file}:",
                f"Total rows: {total_rows}",
                f"Test rows: {test_count} (1%)",
                f"Train rows: {total_rows - test_count} (99%)\n"
            ]
            
            for stat in stats:
                print(stat)
                stats_file.write(stat + '\n')

#### OPTIONAL for other use-cases. The script below is optional and will divide the json files into 2 files (fine-tuneing with openpipe; can upload a file with maximum of 50k rows at a time). The first file will have 50k rows and the other the remaining. IN our case it doesn't apply.

In [ ]:
def check_and_split_large_files(strategy_files, creator, row_limit=50000):
    files_split = False
    
    for file_path in strategy_files:
        # Read the JSONL file
        with open(file_path, 'r') as file:
            jsonl_data = [json.loads(line) for line in file]
        
        total_rows = len(jsonl_data)
        
        if total_rows > row_limit:
            files_split = True
            # Get file name components
            strategy_name = file_path.split('_')[1]  # A, B, or C
            
            # Create file names for splits in creator's directory
            first_file = f'output_files/{creator}/Strategy_{strategy_name}1_{creator}.jsonl'
            second_file = f'output_files/{creator}/Strategy_{strategy_name}2_{creator}.jsonl'
            
            # Split the data
            first_chunk = jsonl_data[:row_limit]
            second_chunk = jsonl_data[row_limit:]
            
            # Save first chunk (up to 50k)
            with open(first_file, 'w') as f:
                for entry in first_chunk:
                    f.write(json.dumps(entry) + '\n')
                    
            # Save second chunk (remaining rows)
            with open(second_file, 'w') as f:
                for entry in second_chunk:
                    f.write(json.dumps(entry) + '\n')
            
            print(f"Split {file_path}:")
            print(f"- {first_file}: {len(first_chunk)} rows")
            print(f"- {second_file}: {len(second_chunk)} rows\n")
    
    if not files_split:
        print(f"None of the files for {creator} need to split (all under 50k rows).")

# Process each creator's files
for creator in dfs.keys():
    print(f"\n=== Checking {creator}'s files for splitting ===")
    
    # List of strategy files for this creator
    strategy_files = [
        f'output_files/{creator}/Strategy_A_one_pair_{creator}.jsonl',
        f'output_files/{creator}/Strategy_B_ten_pairs_{creator}.jsonl',
        f'output_files/{creator}/Strategy_C_rolling_format_{creator}.jsonl'
    ]
    
    # Check and split files if needed
    check_and_split_large_files(strategy_files, creator)

The next step is to fine-tune the models - which in our case happens through OpenPipe (Amazing time and computational resources saver). This can be done through their Web Interface or API. Other methods of fine-tuning such as localy done or through hugging face API are possible and disscused/provided in the thesis.
This part is sensitive but hyperparameter configuration will be shared. While the spotlight of this work relies on how the data is manipulated, consistnecy & reproducibility were part of the main focus.

Discussion: 


Possible counter Questions:
Q: How else would you test the models on the same thing? One cannot test the models on different testsets- it doesn't make any sense. And with the implemented method you have a bias-free dataset- a dataset that contains all the formats.

Q: Maybe you would ask yourself but why dont you keep the initial amount of data, well because the only thing that should influence the performance of the Fine-tuned models should be the strategies that we have employed. We want to make sure that nothing esle might affect (too greatly) the final results.

Q: Why did you trim the data in that point of the pipeline and not somewhere else? 1) where and how do you trim the data? In which point of the pipeline? 2) How do you maintain anyway the same amount of testing data in this case? You would ruin the flow and content of conversations.

Another debate : We could have more models for each iteration of improvmenetns (as we had initially) or we could have only 3 models but with much more testing data. Why? because of the resources it takes to train so many mdoels and also generate the reposnes. But fewer models to train means less money and winning some time, so that time can be fed back into generating responses at least.